In [1]:
from collections import defaultdict
import json
import os
import pickle
import statistics

from requests import Session

import matplotlib.pyplot as plt
import pandas as pd

import statbotics

%matplotlib notebook

In [2]:
AUTH_KEY = "XeUIxlvO4CPc44NlLE3ncevDg7bAhp6CRy6zC9M2aQb2zGfys0M30eKwavFJSEJr"

read_prefix = "https://www.thebluealliance.com/api/v3/"

session = Session()
session.headers.update({"X-TBA-Auth-Key": AUTH_KEY, "X-TBA-Auth-Id": ""})

sb = statbotics.Statbotics()

In [3]:
def dump(path, data):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
        with open(path, "wb") as f:
            pickle.dump(data, f)
    except OSError:
        pass


def load(file):
    with open(file, "rb") as f:
        return pickle.load(f)


def dump_cache(path, data):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
        with open(path + "/data.p", "wb") as f:
            pickle.dump(data, f)
    except OSError:
        pass


def load_cache(file):
    with open(file + "/data.p", "rb") as f:
        return pickle.load(f)

    
    
def get_tba(url, cache=True):
    if cache and os.path.exists("../../backend/cache/" + url + "/data.p"):
        # Cache Hit
        return load_cache("../../backend/cache/" + url)

    response = session.get(read_prefix + url)
    data = response.json()

    # Cache Miss
    dump_cache("../../backend/cache/" + url, data)
    return data

In [4]:
def get_week(key, week):
    if key in ["2023arc", "2023cur", "2023dal", "2023gal", "2023hop", "2023joh", "2023mil", "2023new"]:
        week = 8
    return week

events = get_tba("events/2023")
events = [(e["key"], get_week(e["key"], e["week"])) for e in events]
print(len(events))

all_matches = []
for event, week in events:
    all_matches.extend([x, week] for x in get_tba("event/" + event + "/matches", False))
    
print(len(all_matches))

197
16238


In [5]:
year = sb.get_year(2023)
teams = sb.get_team_years(year=2023, limit=10000)

TOTAL_MEAN, TOTAL_SD = year["score_mean"], year["score_sd"]
starting_epas = {t["team"]: (t["epa_start"] - TOTAL_MEAN / 3) / TOTAL_SD for t in teams}
locations = {t["team"]: (t["country"], t["state"], t["district"]) for t in teams}
print(starting_epas[254], starting_epas[5511])

0.9612851952770209 0.03689827429609454


In [6]:
def process_match(m, w):
    rb = m["score_breakdown"]["red"]
    bb = m["score_breakdown"]["blue"]

    out = {
        "week": w + 1, # TBA off by one
        "event": m["event_key"],
        "key": m["key"],
        "time": m["actual_time"],
        "playoff": m["comp_level"] != "qm",
        "red": [int(x[3:]) for x in m["alliances"]["red"]["team_keys"]],
        "blue": [int(x[3:]) for x in m["alliances"]["blue"]["team_keys"]],
        "red_auto_mobility": rb["autoMobilityPoints"],
        "blue_auto_mobility": bb["autoMobilityPoints"],
        "red_auto_charge_station": rb["autoChargeStationPoints"],
        "blue_auto_charge_station": bb["autoChargeStationPoints"],
        "red_auto_cycles": rb["autoGamePieceCount"],
        "blue_auto_cycles": bb["autoGamePieceCount"],
        "red_auto_points": rb["autoGamePiecePoints"],
        "blue_auto_points": bb["autoGamePiecePoints"],
        "red_teleop_cycles": rb["teleopGamePieceCount"] - rb["autoGamePieceCount"],
        "blue_teleop_cycles": bb["teleopGamePieceCount"] - bb["autoGamePieceCount"],
        "red_teleop_points": rb["teleopGamePiecePoints"],
        "blue_teleop_points": bb["teleopGamePiecePoints"],
        "red_links": rb["linkPoints"] / 5,
        "blue_links": bb["linkPoints"] / 5,
        "red_link_points": rb["linkPoints"],
        "blue_link_points": bb["linkPoints"],
        "red_total_cycles": rb["teleopGamePieceCount"],
        "blue_total_cycles": bb["teleopGamePieceCount"],
        "red_total_points": rb["autoGamePiecePoints"] + rb["teleopGamePiecePoints"] + rb["linkPoints"],
        "blue_total_points":  bb["autoGamePiecePoints"] + bb["teleopGamePiecePoints"] + bb["linkPoints"],
        "red_auto_bot_cubes": len([x for x in rb["autoCommunity"]["B"] if x == "Cube"]),
        "blue_auto_bot_cubes": len([x for x in bb["autoCommunity"]["B"] if x == "Cube"]),
        "red_auto_mid_cubes": len([x for x in rb["autoCommunity"]["M"] if x == "Cube"]),
        "blue_auto_mid_cubes": len([x for x in bb["autoCommunity"]["M"] if x == "Cube"]),
        "red_auto_top_cubes": len([x for x in rb["autoCommunity"]["T"] if x == "Cube"]),
        "blue_auto_top_cubes": len([x for x in bb["autoCommunity"]["T"] if x == "Cube"]),
        "red_auto_bot_cones": len([x for x in rb["autoCommunity"]["B"] if x == "Cone"]),
        "blue_auto_bot_cones": len([x for x in bb["autoCommunity"]["B"] if x == "Cone"]),
        "red_auto_mid_cones": len([x for x in rb["autoCommunity"]["M"] if x == "Cone"]),
        "blue_auto_mid_cones": len([x for x in bb["autoCommunity"]["M"] if x == "Cone"]),
        "red_auto_top_cones": len([x for x in rb["autoCommunity"]["T"] if x == "Cone"]),
        "blue_auto_top_cones": len([x for x in bb["autoCommunity"]["T"] if x == "Cone"]),
        "red_auto_bot_cycles": len([x for x in rb["autoCommunity"]["B"] if x != "None"]),
        "blue_auto_bot_cycles": len([x for x in bb["autoCommunity"]["B"] if x != "None"]),
        "red_auto_mid_cycles": len([x for x in rb["autoCommunity"]["M"] if x != "None"]),
        "blue_auto_mid_cycles": len([x for x in bb["autoCommunity"]["M"] if x != "None"]),
        "red_auto_top_cycles": len([x for x in rb["autoCommunity"]["T"] if x != "None"]),
        "blue_auto_top_cycles": len([x for x in bb["autoCommunity"]["T"] if x != "None"]),
        "red_endgame_charge_station": rb["endGameChargeStationPoints"],
        "blue_endgame_charge_station": bb["endGameChargeStationPoints"],
        "red_fouls_committed": bb["foulPoints"],
        "blue_fouls_committed": rb["foulPoints"],
        "red_fouls_drawn": rb["foulPoints"],
        "blue_fouls_drawn": bb["foulPoints"],
        "red_capped_fouls_committed": min(20, bb["foulPoints"]),
        "blue_capped_fouls_committed": min(20, rb["foulPoints"]),
        "red_capped_fouls_drawn": min(20, rb["foulPoints"]),
        "blue_capped_fouls_drawn": min(20, bb["foulPoints"]),
    }
    
    out["red_teleop_bot_cubes"] = len([x for x in rb["teleopCommunity"]["B"] if x == "Cube"]) - out["red_auto_bot_cubes"]
    out["blue_teleop_bot_cubes"] = len([x for x in bb["teleopCommunity"]["B"] if x == "Cube"]) - out["blue_auto_bot_cubes"]
    out["red_teleop_mid_cubes"] = len([x for x in rb["teleopCommunity"]["M"] if x == "Cube"]) - out["red_auto_mid_cubes"]
    out["blue_teleop_mid_cubes"] = len([x for x in bb["teleopCommunity"]["M"] if x == "Cube"]) - out["blue_auto_mid_cubes"]
    out["red_teleop_top_cubes"] = len([x for x in rb["teleopCommunity"]["T"] if x == "Cube"]) - out["red_auto_top_cubes"]
    out["blue_teleop_top_cubes"] = len([x for x in bb["teleopCommunity"]["T"] if x == "Cube"]) - out["blue_auto_top_cubes"]
    out["red_teleop_bot_cones"] = len([x for x in rb["teleopCommunity"]["B"] if x == "Cone"]) - out["red_auto_bot_cones"]
    out["blue_teleop_bot_cones"] = len([x for x in bb["teleopCommunity"]["B"] if x == "Cone"]) - out["blue_auto_bot_cones"]
    out["red_teleop_mid_cones"] = len([x for x in rb["teleopCommunity"]["M"] if x == "Cone"]) - out["red_auto_mid_cones"]
    out["blue_teleop_mid_cones"] = len([x for x in bb["teleopCommunity"]["M"] if x == "Cone"]) - out["blue_auto_mid_cones"]
    out["red_teleop_top_cones"] = len([x for x in rb["teleopCommunity"]["T"] if x == "Cone"]) - out["red_auto_top_cones"]
    out["blue_teleop_top_cones"] = len([x for x in bb["teleopCommunity"]["T"] if x == "Cone"]) - out["blue_auto_top_cones"]
    out["red_teleop_bot_cycles"] = len([x for x in rb["teleopCommunity"]["B"] if x != "None"]) - out["red_auto_bot_cycles"]
    out["blue_teleop_bot_cycles"] = len([x for x in bb["teleopCommunity"]["B"] if x != "None"]) - out["blue_auto_bot_cycles"]
    out["red_teleop_mid_cycles"] = len([x for x in rb["teleopCommunity"]["M"] if x != "None"]) - out["red_auto_mid_cycles"]
    out["blue_teleop_mid_cycles"] = len([x for x in bb["teleopCommunity"]["M"] if x != "None"]) - out["blue_auto_mid_cycles"]
    out["red_teleop_top_cycles"] = len([x for x in rb["teleopCommunity"]["T"] if x != "None"]) - out["red_auto_top_cycles"]
    out["blue_teleop_top_cycles"] = len([x for x in bb["teleopCommunity"]["T"] if x != "None"]) - out["blue_auto_top_cycles"]
    
    out["red_auto_cube_cycles"] = out["red_auto_bot_cubes"] + out["red_auto_mid_cubes"] + out["red_auto_top_cubes"]
    out["blue_auto_cube_cycles"] = out["blue_auto_bot_cubes"] + out["blue_auto_mid_cubes"] + out["blue_auto_top_cubes"]
    out["red_auto_cone_cycles"] = out["red_auto_bot_cones"] + out["red_auto_mid_cones"] + out["red_auto_top_cones"]
    out["blue_auto_cone_cycles"] = out["blue_auto_bot_cones"] + out["blue_auto_mid_cones"] + out["blue_auto_top_cones"]
    out["red_auto_cube_points"] = 3 * out["red_auto_bot_cubes"] + 4 * out["red_auto_mid_cubes"] + 6 * out["red_auto_top_cubes"]
    out["blue_auto_cube_points"] = 3 * out["blue_auto_bot_cubes"] + 4 * out["blue_auto_mid_cubes"] + 6 * out["blue_auto_top_cubes"]
    out["red_auto_cone_points"] = 3 * out["red_auto_bot_cones"] + 4 * out["red_auto_mid_cones"] + 6 * out["red_auto_top_cones"]
    out["blue_auto_cone_points"] = 3 * out["blue_auto_bot_cones"] + 4 * out["blue_auto_mid_cones"] + 6 * out["blue_auto_top_cones"]
    
    out["red_teleop_cube_cycles"] = out["red_teleop_bot_cubes"] + out["red_teleop_mid_cubes"] + out["red_teleop_top_cubes"]
    out["blue_teleop_cube_cycles"] = out["blue_teleop_bot_cubes"] + out["blue_teleop_mid_cubes"] + out["blue_teleop_top_cubes"]
    out["red_teleop_cone_cycles"] = out["red_teleop_bot_cones"] + out["red_teleop_mid_cones"] + out["red_teleop_top_cones"]
    out["blue_teleop_cone_cycles"] = out["blue_teleop_bot_cones"] + out["blue_teleop_mid_cones"] + out["blue_teleop_top_cones"]
    out["red_teleop_cube_points"] = 2 * out["red_teleop_bot_cubes"] + 3 * out["red_teleop_mid_cubes"] + 5 * out["red_teleop_top_cubes"]
    out["blue_teleop_cube_points"] = 2 * out["blue_teleop_bot_cubes"] + 3 * out["blue_teleop_mid_cubes"] + 5 * out["blue_teleop_top_cubes"]
    out["red_teleop_cone_points"] = 2 * out["red_teleop_bot_cones"] + 3 * out["red_teleop_mid_cones"] + 5 * out["red_teleop_top_cones"]
    out["blue_teleop_cone_points"] = 2 * out["blue_teleop_bot_cones"] + 3 * out["blue_teleop_mid_cones"] + 5 * out["blue_teleop_top_cones"]
    
    out["red_cube_cycles"] = out["red_auto_cube_cycles"] + out["red_teleop_cube_cycles"]
    out["blue_cube_cycles"] = out["blue_auto_cube_cycles"] + out["blue_teleop_cube_cycles"]
    out["red_cone_cycles"] = out["red_auto_cone_cycles"] + out["red_teleop_cone_cycles"]
    out["blue_cone_cycles"] = out["blue_auto_cone_cycles"] + out["blue_teleop_cone_cycles"]
    out["red_cube_points"] = out["red_auto_cube_points"] + out["red_teleop_cube_points"]
    out["blue_cube_points"] = out["blue_auto_cube_points"] + out["blue_teleop_cube_points"]
    out["red_cone_points"] = out["red_auto_cone_points"] + out["red_teleop_cone_points"]
    out["blue_cone_points"] = out["blue_auto_cone_points"] + out["blue_teleop_cone_points"]
    
    out["red_auto_bot_cycles"] = out["red_auto_bot_cubes"] + out["red_auto_bot_cones"]
    out["blue_auto_bot_cycles"] = out["blue_auto_bot_cubes"] + out["blue_auto_bot_cones"]
    out["red_auto_mid_cycles"] = out["red_auto_mid_cubes"] + out["red_auto_mid_cones"]
    out["blue_auto_mid_cycles"] = out["blue_auto_mid_cubes"] + out["blue_auto_mid_cones"]
    out["red_auto_top_cycles"] = out["red_auto_top_cubes"] + out["red_auto_top_cones"]
    out["blue_auto_top_cycles"] = out["blue_auto_top_cubes"] + out["blue_auto_top_cones"]
    out["red_auto_bot_points"] = 3 * out["red_auto_bot_cycles"]
    out["blue_auto_bot_points"] = 3 * out["blue_auto_bot_cycles"]
    out["red_auto_mid_points"] = 4 * out["red_auto_mid_cycles"]
    out["blue_auto_mid_points"] = 4 * out["blue_auto_mid_cycles"]
    out["red_auto_top_points"] = 6 * out["red_auto_top_cycles"]
    out["blue_auto_top_points"] = 6 * out["blue_auto_top_cycles"]
    
    out["red_teleop_bot_cycles"] = out["red_teleop_bot_cubes"] + out["red_teleop_bot_cones"]
    out["blue_teleop_bot_cycles"] = out["blue_teleop_bot_cubes"] + out["blue_teleop_bot_cones"]
    out["red_teleop_mid_cycles"] = out["red_teleop_mid_cubes"] + out["red_teleop_mid_cones"]
    out["blue_teleop_mid_cycles"] = out["blue_teleop_mid_cubes"] + out["blue_teleop_mid_cones"]
    out["red_teleop_top_cycles"] = out["red_teleop_top_cubes"] + out["red_teleop_top_cones"]
    out["blue_teleop_top_cycles"] = out["blue_teleop_top_cubes"] + out["blue_teleop_top_cones"]
    out["red_teleop_bot_points"] = 2 * out["red_teleop_bot_cycles"]
    out["blue_teleop_bot_points"] = 2 * out["blue_teleop_bot_cycles"]
    out["red_teleop_mid_points"] = 3 * out["red_teleop_mid_cycles"]
    out["blue_teleop_mid_points"] = 3 * out["blue_teleop_mid_cycles"]
    out["red_teleop_top_points"] = 5 * out["red_teleop_top_cycles"]
    out["blue_teleop_top_points"] = 5 * out["blue_teleop_top_cycles"]
    
    out["red_bot_cycles"] = out["red_auto_bot_cycles"] + out["red_teleop_bot_cycles"]
    out["blue_bot_cycles"] = out["blue_auto_bot_cycles"] + out["blue_teleop_bot_cycles"]
    out["red_mid_cycles"] = out["red_auto_mid_cycles"] + out["red_teleop_mid_cycles"]
    out["blue_mid_cycles"] = out["blue_auto_mid_cycles"] + out["blue_teleop_mid_cycles"]
    out["red_top_cycles"] = out["red_auto_top_cycles"] + out["red_teleop_top_cycles"]
    out["blue_top_cycles"] = out["blue_auto_top_cycles"] + out["blue_teleop_top_cycles"]
    out["red_bot_points"] = out["red_auto_bot_points"] + out["red_teleop_bot_points"]
    out["blue_bot_points"] = out["blue_auto_bot_points"] + out["blue_teleop_bot_points"]
    out["red_mid_points"] = out["red_auto_mid_points"] + out["red_teleop_mid_points"]
    out["blue_mid_points"] = out["blue_auto_mid_points"] + out["blue_teleop_mid_points"]
    out["red_top_points"] = out["red_auto_top_points"] + out["red_teleop_top_points"]
    out["blue_top_points"] = out["blue_auto_top_points"] + out["blue_teleop_top_points"]
    
    return out

processed_matches = sorted([process_match(m, w) for m, w in all_matches if m["actual_time"] is not None and m["score_breakdown"] is not None and w is not None], key=lambda x: x["time"])
week_one_matches = [m for m in processed_matches if m["week"] == 1]

In [7]:
for match in processed_matches:
    assert match["red_bot_cycles"] + match["red_mid_cycles"] + match["red_top_cycles"] == match["red_total_cycles"]
    assert match["blue_bot_cycles"] + match["blue_mid_cycles"] + match["blue_top_cycles"] == match["blue_total_cycles"]

In [8]:
keys = [
    "auto_cycles",
    "auto_points",
    "teleop_cycles",
    "teleop_points",
    "total_cycles",
    "links",
    "link_points",
    "total_points",
    "cube_cycles",
    "cube_points",
    "cone_cycles",
    "cone_points",
    "bot_cycles",
    "bot_points",
    "mid_cycles",
    "mid_points",
    "top_cycles",
    "top_points",
    "fouls_committed",
    "fouls_drawn",
    "capped_fouls_committed",
    "capped_fouls_drawn",
]

all_keys = [
    "auto_mobility",
    "auto_charge_station",
    "auto_cycles",
    "auto_points",
    "teleop_cycles",
    "teleop_points",
    "total_cycles",
    "links",
    "link_points",
    "total_points",
    "cube_cycles",
    "cube_points",
    "cone_cycles",
    "cone_points",
    "bot_cycles",
    "bot_points",
    "mid_cycles",
    "mid_points",
    "top_cycles",
    "top_points",
    "auto_bot_cubes",
    "auto_mid_cubes",
    "auto_top_cubes",
    "auto_bot_cones",
    "auto_mid_cones",
    "auto_top_cones",
    "auto_bot_cycles",
    "auto_mid_cycles",
    "auto_top_cycles",
    "auto_cube_cycles",
    "auto_cone_cycles",
    "teleop_bot_cubes",
    "teleop_mid_cubes",
    "teleop_top_cubes",
    "teleop_bot_cones",
    "teleop_mid_cones",
    "teleop_top_cones",
    "teleop_bot_cycles",
    "teleop_mid_cycles",
    "teleop_top_cycles",
    "teleop_cube_cycles",
    "teleop_cone_cycles",
    "endgame_charge_station",
    "fouls_committed",
    "fouls_drawn",
    "capped_fouls_committed",
    "capped_fouls_drawn",
]

foul_keys = ["fouls_committed", "fouls_drawn", "capped_fouls_committed", "capped_fouls_drawn"]

team_epas = defaultdict(dict)

for key in all_keys:
    stats = [m["red_" + key] for m in week_one_matches] + [m["blue_" + key] for m in week_one_matches]
    mean, sd = sum(stats) / len(stats), statistics.stdev(stats)

    for team in starting_epas:
        if key in foul_keys:
            team_epas[team][key] = [0]
        else:
            team_epas[team][key] = [mean / 3 + sd * starting_epas[team]]

# Normalize auto/teleop, bot/mid/top, cube/cone to sum to total, keeping original ratio
for team in team_epas:
    for components, total in [
        (("auto_cycles", "teleop_cycles"), "total_cycles"),
        (("auto_points", "teleop_points", "link_points"), "total_points"),
        (("cube_cycles", "cone_cycles"), "total_cycles"),
        (("cube_points", "cone_points", "link_points"), "total_points"),
        (("bot_cycles", "mid_cycles", "top_cycles"), "total_cycles"),
        (("bot_points", "mid_points", "top_points", "link_points"), "total_points"),
    ]:
        comp_sum = sum([max(team_epas[team][key][-1], 0) for key in components if key != "link_points"])
        total_sum = team_epas[team][total][-1] - (team_epas[team]["link_points"][-1] if "link_points" in components else 0)
        for component in components:
            if component != "link_points":
                team_epas[team][component][-1] *= total_sum / max(comp_sum, 1)
        
team_event_epas = defaultdict(dict)
        
for key in all_keys:
    for match in processed_matches:
        red_teams = match["red"]
        blue_teams = match["blue"]

        if max(max(red_teams), max(blue_teams)) > 9500:
            continue
            
        for team in red_teams + blue_teams:
            team_event_key = str(team) + "_" + match["event"]
            if key not in team_event_epas[team_event_key]:
                team_event_epas[team_event_key][key] = [team_epas[team][key][-1]]

        red_pred = sum(team_epas[x][key][-1] for x in red_teams)
        blue_pred = sum(team_epas[x][key][-1] for x in blue_teams)

        red_actual = match["red_" + key]
        blue_actual = match["blue_" + key]

        weight = 1/3 if match["playoff"] else 1

        red_error = red_actual - red_pred
        for team in red_teams:
            percent = min(0.5, max(0.3, 0.5 - 0.2 / 6 * (len(team_epas[team][key]) - 6)))
            new_value = team_epas[team][key][-1] + weight * percent * red_error / 3
            if key in foul_keys:
                new_value = max(new_value, 0)
            team_epas[team][key].append(new_value)
            team_event_epas[str(team) + "_" + match["event"]][key].append(new_value)

        blue_error = blue_actual - blue_pred
        for team in blue_teams:
            percent = min(0.5, max(0.3, 0.5 - 0.2 / 6 * (len(team_epas[team][key]) - 6)))
            new_value = team_epas[team][key][-1] + weight * percent * blue_error / 3
            if key in foul_keys:
                new_value = max(new_value, 0)
            team_epas[team][key].append(new_value)
            team_event_epas[str(team) + "_" + match["event"]][key].append(new_value)
        
end_epas = {k: {} for k in team_epas}
for k, v in team_epas.items():
    for k2, v2 in v.items():
        end_epas[k][k2] = v2[-1]
        
end_event_epas = {k: {} for k in team_event_epas}
for k, v in team_event_epas.items():
    for k2, v2 in v.items():
        end_event_epas[k][k2] = v2[-1]
    
# print(end_epas[254])
# print(end_event_epas["254_2023cada"])

In [9]:
df = pd.DataFrame(end_epas).T
df[keys].to_json("../../data/2023/epa_breakdown.json", orient="index")
df.to_csv("../../data/2023/epa_breakdown_full.csv")

In [10]:
percentile_stats = {}
for key in all_keys:
    if key in ["fouls_committed", "capped_fouls_committed"]:
        percentile_stats[key] = {
            "p99": df[key].quantile(0.01),
            "p95": df[key].quantile(0.05),
            "p90": df[key].quantile(0.10),
            "p75": df[key].quantile(0.25),
            "p50": df[key].quantile(0.50),
            "p25": df[key].quantile(0.75),
            "mean": df[key].mean(),
            "sd": df[key].std()
        }
    else:
        percentile_stats[key] = {
            "p99": df[key].quantile(0.99),
            "p95": df[key].quantile(0.95),
            "p90": df[key].quantile(0.90),
            "p75": df[key].quantile(0.75),
            "p50": df[key].quantile(0.50),
            "p25": df[key].quantile(0.25),
            "mean": df[key].mean(),
            "sd": df[key].std()
        }
    
percentile_df = pd.DataFrame(percentile_stats).T
percentile_df.to_json("../../data/2023/epa_breakdown_percentiles.json", orient="index")

In [11]:
# event_df = pd.DataFrame(end_event_epas).T
# event_df.to_json("../../frontend/public/data/event_epa_breakdown.json", orient="index")